<a href="https://colab.research.google.com/github/pevu97/Machine-Learning-Kagle/blob/main/Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Housing Prices Competition



Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

### Imports

In [57]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Data import


In [58]:
train = pd.read_csv(r'train.csv', index_col='Id').select_dtypes(include=[float, int])
to_predict = pd.read_csv(r'test.csv',index_col='Id').select_dtypes(include=[float, int])

### Data Analysis

In [59]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
X.isnull().sum().sort_values(ascending=False).head()

,0
LotFrontage,259
GarageYrBlt,81
MasVnrArea,8
WoodDeckSF,0
BedroomAbvGr,0


In [61]:
X.corrwith(y).sort_values(ascending=False).head(20)

,0
OverallQual,0.790982
GrLivArea,0.708624
GarageCars,0.640409
GarageArea,0.623431
TotalBsmtSF,0.613581
1stFlrSF,0.605852
FullBath,0.560664
TotRmsAbvGrd,0.533723
YearBuilt,0.522897
YearRemodAdd,0.507101


In [62]:
columns = X.corrwith(y).sort_values(ascending=False)[:13].index.tolist()
X = X[columns]
to_predict = to_predict[columns]

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

to_predict_scaled = scaler.transform(to_predict)

In [65]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=42, n_estimators=200, max_depth=3)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))

0.9850166440010071


In [66]:
model.score(X_test_scaled, y_test)

0.8495686054229736

In [67]:
y_predict = model.predict(to_predict)

predict = pd.DataFrame(y_predict, columns=['SalePrice'])
predict.index = to_predict.index


predict.to_csv('predict.csv', index=True)